## 1.Make the necessary imports:

In [46]:
import librosa
import soundfile as sf
import os,glob,pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


### 2.Define a function extract_feature to extract the mfcc, chroma, and mel features from a sound file. This function takes 4 parameters- the file  name and three Boolean parameters for the three features:

### mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound
### chroma: Pertains to the 12 different pitch classes
### mel: Mel Spectrogram Frequency

In [47]:
def extract_feature(file_name,mfcc,chroma,mel):
    with sf.SoundFile(file_name) as sound_file:
        X= sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=40).T,axis=0)
            result=np.hstack((result,mfccs))
        if chroma:

            chroma=np.mean(librosa.feature.chroma_stft(S=stft,sr=sample_rate).T,axis=0)
            result=np.hstack((result,chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0)
            result=np.hstack((result,mel))
    return result
    

## 3.Define a dictionary to hold numbers and the emotions available in the RAVDESS dataset, and a list to hold those we want- calm, happy, fearful, disgust.

In [48]:
emotions={
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised'
}

observed_emotions=['calm','happy','fearful','disgust']

##  4.Load the data and extract features for each sound file

In [49]:
def load_data(test_size=0.2):
    x, y = [], []
    for file in glob.glob("/kaggle/input/speech-emotion-dataset-zip/Actor_*/*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions.get(file_name.split("-")[2])
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        if feature is not None:  # Skip bad files
            x.append(feature)
            y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)


## 5.Split the dataset

In [50]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

## 6.Get the shape of the training and testing datasets

In [51]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


## 7.Get the number of features extracted

In [52]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


## 8.Initialize the Multi Layer Perceptron Classifier

In [53]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

## 9.Train the model

In [54]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

## 10.Predict for the test set

In [55]:
y_pred=model.predict(x_test)

## 11.Calculate the accuracy of our model

In [56]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 71.35%
